# Random Forest

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv('../data/tackles.csv').groupby(['gameId', 'playId']).sum()
df = df.join(
    other = pd.read_csv('../data/plays.csv').set_index(['gameId', 'playId']),
    on = ['gameId', 'playId'],
    how = 'inner'
)
df.head()

nflId  tackle  assist  forcedFumble  pff_missedTackle  \
gameId     playId                                                          
2022090800 56      43294       1       0             0                 0   
           80      53532       1       0             0                 0   
           101     42816       1       0             0                 0   
           122     38577       1       0             0                 0   
           146     81912       0       2             0                 0   

                   ballCarrierId ballCarrierDisplayName  \
gameId     playId                                         
2022090800 56              42489           Stefon Diggs   
           80              46076             Josh Allen   
           101             47857       Devin Singletary   
           122             47857       Devin Singletary   
           146             47857       Devin Singletary   

                                                     playDescription  quarter  \
gameId     playId                                                               
2022090800 56      (15:00) (Shotgun) J.Allen pass short right to ...        1   
           80      (14:29) (No Huddle, Shotgun) J.Allen scrambles...        1   
           101     (13:54) D.Singletary right end to BUF 45 for 7...        1   
           122     (13:15) (Shotgun) J.Allen pass short right to ...        1   
           146     (12:33) (Shotgun) D.Singletary left tackle to ...        1   

                   down  ...  preSnapHomeTeamWinProbability  \
gameId     playId        ...                                  
2022090800 56         1  ...                       0.413347   
           80         2  ...                       0.413316   
           101        1  ...                       0.399819   
           122        2  ...                       0.384969   
           146        1  ...                       0.365938   

                  preSnapVisitorTeamWinProbability  \
gameId     playId                                    
2022090800 56                             0.586653   
           80                             0.586684   
           101                            0.600181   
           122                            0.615031   
           146                            0.634062   

                  homeTeamWinProbabilityAdded visitorTeamWinProbilityAdded  \
gameId     playId                                                            
2022090800 56                       -0.000031                     0.000031   
           80                       -0.013497                     0.013497   
           101                      -0.014850                     0.014850   
           122                      -0.019032                     0.019032   
           146                       0.021616                    -0.021616   

                   expectedPoints expectedPointsAdded  foulName1  foulName2  \
gameId     playId                                                             
2022090800 56            1.298699            0.004420        NaN        NaN   
           80            1.303119            0.823571        NaN        NaN   
           101           2.126690            0.562363        NaN        NaN   
           122           2.689053            0.399209        NaN        NaN   
           146           3.088262           -0.514056        NaN        NaN   

                  foulNFLId1  foulNFLId2  
gameId     playId                         
2022090800 56            NaN         NaN  
           80            NaN         NaN  
           101           NaN         NaN  
           122           NaN         NaN  
           146           NaN         NaN  

[5 rows x 38 columns]

In [3]:
df_cleaned = df[[
    'quarter',
    'down',
    'possessionTeam',
    'yardsToGo',
    'yardlineSide',
    'yardlineNumber',
    'offenseFormation',
    'defendersInTheBox',
    'pff_missedTackle'
]]
df_cleaned.isna().sum()

quarter                0
down                   0
possessionTeam         0
yardsToGo              0
yardlineSide         164
yardlineNumber         0
offenseFormation       4
defendersInTheBox      5
pff_missedTackle       0
dtype: int64

A small number of plays without `offenseFormation` or `defendersInTheBox` observations will be dropped.

In [4]:
df_cleaned.loc[:, 'yardlineSide'] = df_cleaned.apply(lambda x:
    # 0 = Midfield
    0 if x['yardlineSide'] == 'NA'
    # 1 = Own territory
    else 1 if x['yardlineSide'] == x['possessionTeam']
    # 2 = Opp territory
    else 2,
axis=1)
df_cleaned.loc[:, 'offenseFormation'] = df_cleaned['offenseFormation'].map({
    'EMPTY': 1,
    'I_FORM': 2,
    'JUMBO': 3,
    'PISTOL': 4,
    'SHOTGUN': 5,
    'SINGLEBACK': 6,
    'WILDCAT': 7
})
df_cleaned = df_cleaned.dropna()

In [5]:
# Select relevant features and target variable
# The features should be things we know at the start of the play, to predict whether a tackle could be missed during the play.
features = df_cleaned[['quarter', 'down', 'yardsToGo', 'yardlineNumber', 'yardlineSide', 'offenseFormation', 'defendersInTheBox']]
target = df_cleaned['pff_missedTackle'].clip(0, 1)
features

quarter  down  yardsToGo  yardlineNumber yardlineSide  \
gameId     playId                                                          
2022090800 56            1     1         10              25            1   
           80            1     2          4              31            1   
           101           1     1         10              38            1   
           122           1     2          3              45            1   
           146           1     1         10              49            2   
...                    ...   ...        ...             ...          ...   
2022110700 3658          4     3          1              44            1   
           3686          4     1         10              46            1   
           3707          4     2          5              49            2   
           3740          4     3          1              45            2   
           3787          4     1         10              11            1   

                  offenseFormation  defendersInTheBox  
gameId     playId                                      
2022090800 56                  5.0                6.0  
           80                  1.0                6.0  
           101                 2.0                6.0  
           122                 5.0                6.0  
           146                 5.0                6.0  
...                            ...                ...  
2022110700 3658                4.0                7.0  
           3686                2.0                8.0  
           3707                4.0                9.0  
           3740                4.0               10.0  
           3787                5.0                4.0  

[12020 rows x 7 columns]

In [6]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [7]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(x_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy: .2f}')
print('\nClassification Report:\n', classification_rep)

Accuracy:  0.83

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.96      0.90      2048
           1       0.20      0.06      0.09       356

    accuracy                           0.83      2404
   macro avg       0.53      0.51      0.50      2404
weighted avg       0.76      0.83      0.78      2404

